# KBO 선수 크롤링
https://www.koreabaseball.com/Player/Search.aspx

In [2]:
!pip install selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np
import requests

     |████████████████████████████████| 911kB 31.6MB/s 


In [3]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [5]:
column = []
def Player(team):
    players = []
    remove = []
    temp_list = []
    temp_url = [] 
    time.sleep(1.5)
    url = driver.get('https://www.koreabaseball.com/Player/Search.aspx')        # Player 전체 URL
    time.sleep(1.5)
    driver.find_element_by_class_name('select02').click()
    time.sleep(1.5)
    xpath_format='//option[@value="'+team+'"]'
    driver.find_element_by_xpath(xpath_format).click()
    time.sleep(1.5)
    driver.find_element_by_id(f'cphContents_cphContents_cphContents_ucPager_btnNo{1}').click()
    time.sleep(1.5)
    team_player = driver.find_element_by_class_name('point').text               # 팀별 선수 수
    time.sleep(1.5)
    count = int(team_player)
    page = (count-1) // 20 + 1
    print(count)
    for j in range(1, page + 1):
        time.sleep(2.5)
        driver.find_element_by_id(f'cphContents_cphContents_cphContents_ucPager_btnNo{j}').click()
        time.sleep(2.5)
        temp_list = [x.split(' ') for x in driver.find_element_by_tag_name('table').text.split('\n')]
        column = temp_list[0]
        temp_list.pop(0)
        temp_list =  [item for item in temp_list if item[0].isdigit()]
        print(temp_list)
        for k in range(len(temp_list)):
            temp_url = driver.find_element_by_link_text(temp_list[k][1]).get_attribute("href")
            time.sleep(2.5)
            temp_list[k].append(temp_url)
        players += temp_list
    return players

In [6]:
def URL(player):
    URL = []
    for i in range(len(player)):
        URL.append(player[i][-1])
    return URL

In [7]:
def player_info(player):
    player_info = []
    for i in range(len(player)):
        if 'Futures' in player[i]:
            league = 2
        else : 
            league = 1
        url = player[i]
        html = requests.get(url)
        bs_html = BeautifulSoup(html.content,"html.parser")
        if league == 1:
            tuta_type = (bs_html.find("span",{"id":"cphContents_cphContents_cphContents_playerProfile_lblPosition"})).text
            tuta_type = tuta_type[-5:-1]
            payment= (bs_html.find("span", {"id":"cphContents_cphContents_cphContents_playerProfile_lblPayment"})).text
            salary = (bs_html.find("span", {"id":"cphContents_cphContents_cphContents_playerProfile_lblSalary"})).text
            debut = (bs_html.find("span", {"id":"cphContents_cphContents_cphContents_playerProfile_lblJoinInfo"})).text
        else :
            tuta_type = bs_html.find("span",{"id":"cphContents_cphContents_cphContents_ucPlayerProfile_lblPosition"}).text
            tuta_type = tuta_type[-5:-1]

            payment= (bs_html.find("span", {"id":"cphContents_cphContents_cphContents_ucPlayerProfile_lblPayment"})).text
            salary = (bs_html.find("span", {"id":"cphContents_cphContents_cphContents_ucPlayerProfile_lblSalary"})).text
            debut = (bs_html.find("span", {"id":"cphContents_cphContents_cphContents_ucPlayerProfile_lblJoinInfo"})).text
        player_info.append([url, league, tuta_type, payment, salary, debut])
    return player_info

In [33]:
def to_DF(player, info):
    player = pd.DataFrame(player)
    info = pd.DataFrame(info)
    player = player.iloc[:,0:7]
    player.columns = ["number", "이름", "팀","pos","birth","key","weight"]
    info.columns = ["url", "league", "tuta_type","payment","salary","debut"]
    return pd.concat([player,info], axis=1, join='inner')

In [45]:
NC_player = Player("NC")
NC_URL = URL(NC_player)
NC_info = player_info(NC_URL)
NC = to_DF(NC_player, NC_info)

83
[['21', '강동연', 'NC', '투수', '1992-12-18', '195cm,', '94kg', '진북초-덕수중-유신고-두산-상무-두산'], ['68', '강윤구', 'NC', '투수', '1990-07-10', '183cm,', '89kg', '서울이수초-서울경원중-장충고-히어로즈-넥센-상무-넥센'], ['49', '강진성', 'NC', '내야수', '1993-10-19', '180cm,', '81kg', '가동초-잠신중-경기고-NC-경찰'], ['78', '강태경', 'NC', '투수', '2001-07-26', '188cm,', '95kg', '양덕초-잠실중-배명고'], ['59', '구창모', 'NC', '투수', '1997-02-17', '183cm,', '85kg', '천안남산초-덕수중-울산공고'], ['65', '권법수', 'NC', '외야수', '1997-04-05', '183cm,', '82kg', '선학초-재능중-제물포고'], ['36', '권희동', 'NC', '외야수', '1990-12-30', '177cm,', '85kg', '동천초-경주중-경주고-경남대-NC-상무'], ['24', '김건태', 'NC', '투수', '1991-10-02', '185cm,', '85kg', '광주수창초-무등중-진흥고-넥센-상무-넥센'], ['8', '김기환', 'NC', '외야수', '1995-08-25', '182cm,', '74kg', '관산초-대송중-소래고-삼성'], ['5', '김민수', 'NC', '내야수', '1998-07-16', '180cm,', '83kg', '김해삼성초-개성중-부산고'], ['69', '김범준', 'NC', '외야수', '2000-04-20', '183cm,', '90kg', '울산대현초-제일중-대구고'], ['58', '김영규', 'NC', '투수', '2000-02-10', '188cm,', '86kg', '광주서석초-무등중-광주제일고'], ['57', '김영중', 'NC', '투수', '1998-07-

In [12]:
OB_player = Player("OB")
OB_URL = URL(OB_player)
OB_info = player_info(OB_URL)
OB = to_DF(OB_player, OB_info)

80
[['25', '강승호', '두산', '내야수', '1994-02-09', '178cm,', '88kg', '순천북초-천안북중-북일고-LG-경찰-LG-SK'], ['97', '강원진', '두산', '투수', '2002-03-13', '190cm,', '104kg', '연현초-원당중-백송고'], ['69', '강현구', '두산', '외야수', '2002-06-16', '186cm,', '98kg', '서울도림초(인천남동구리틀)-동산중-인천고'], ['58', '고봉재', '두산', '투수', '1993-05-14', '185cm,', '86kg', '감천초-대동중-경남고-호원대'], ['47', '곽빈', '두산', '투수', '1999-05-28', '187cm,', '95kg', '서울학동초-자양중-배명고'], ['15', '국해성', '두산', '외야수', '1989-10-08', '180cm,', '94kg', '군산중앙초-동인천중-인천고'], ['7', '권민석', '두산', '내야수', '1999-02-20', '184cm,', '74kg', '영랑초(수원영통리틀)-설악중-강릉고'], ['38', '권휘', '두산', '투수', '2000-12-07', '181cm,', '87kg', '서울대림초-강남중-덕수고'], ['27', '김강률', '두산', '투수', '1988-08-28', '187cm,', '95kg', '문촌초(일산리틀)-장성중-경기고-두산-상무'], ['92', '김도경', '두산', '외야수', '1997-07-19', '178cm,', '93kg', '율하초-경상중-대구상원고-한양대'], ['94', '김도윤', '두산', '투수', '2002-06-28', '181cm,', '83kg', '신도초(계룡시리틀)-현도중-청주고'], ['30', '김동주', '두산', '투수', '2002-02-14', '190cm,', '90kg', '갈산초-양천중-선린인터넷고'], ['46', '김명신', '두산', '투수', '1993-1

In [20]:
KT_player = Player("KT")
KT_URL = URL(KT_player)
KT_info = player_info(KT_URL)
KT = to_DF(KT_player, KT_info)

86
[['7', '강민국', 'KT', '내야수', '1992-01-10', '176cm,', '80kg', '송정동초-충장중-광주제일고-동국대-NC-상무-NC'], ['64', '강민성', 'KT', '내야수', '1999-12-08', '180cm,', '85kg', '대구옥산초-경상중-경북고'], ['50', '강백호', 'KT', '내야수', '1999-07-29', '184cm,', '98kg', '부천북초-서울이수중-서울고'], ['93', '강승훈', 'KT', '내야수', '1994-01-03', '172cm,', '76kg', '수원신곡초-수원북중-유신고-연세대'], ['66', '강현우', 'KT', '포수', '2001-04-13', '180cm,', '90kg', '원종초(부천시리틀)-부천중-유신고'], ['110', '고영찬', 'KT', '투수', '1999-08-14', '183cm,', '78kg', '상인천초-동인천중-인천고-강릉영동대'], ['1', '고영표', 'KT', '투수', '1991-09-16', '187cm,', '88kg', '광주대성초-광주동성중-화순고-동국대'], ['3', '권동진', 'KT', '내야수', '1998-09-12', '182cm,', '86kg', '제주신광초-세광중-세광고-원광대'], ['0', '김건형', 'KT', '외야수', '1996-07-12', '182cm,', '83kg', '먼우금초-Lesbois(중)-Timberline(고)-Boise', 'State(대)'], ['112', '김대현', 'KT', '내야수', '1998-01-13', '183cm,', '86kg', '서울이수초-서울이수중-장충고-홍익대'], ['37', '김도현', 'KT', '외야수', '1992-10-23', '181cm,', '98kg', '광주서석초-광주동성중-진흥고-넥센-SK-경찰-SK-두산'], ['44', '김만수', 'KT', '포수', '1996-04-18', '180cm,', '80kg'

In [23]:
LG_player = Player("LG")
LG_URL = URL(LG_player)
LG_info = player_info(LG_URL)
LG = to_DF(LG_player, LG_info)

82
[['105', '강민', 'LG', '투수', '2001-04-20', '188cm,', '88kg', '수유초-홍은중-서울고'], ['43', '강효종', 'LG', '투수', '2002-10-14', '184cm,', '86kg', '저동초(일산서구리틀)-충암중-충암고'], ['19', '고우석', 'LG', '투수', '1998-08-06', '182cm,', '90kg', '갈산초-양천중-충암고'], ['15', '고효준', 'LG', '투수', '1983-02-08', '179cm,', '74kg', '서원초-세광중-세광고-롯데-SK-KIA-롯데'], ['2', '구본혁', 'LG', '내야수', '1997-01-11', '177cm,', '75kg', '중대초-잠신중-장충고-동국대'], ['122', '김기연', 'LG', '포수', '1997-09-07', '178cm,', '95kg', '광주수창초-진흥중-진흥고'], ['69', '김대유', 'LG', '투수', '1991-05-08', '187cm,', '92kg', '부산중앙초-부산중-부산고-넥센-SK-KT'], ['45', '김대현', 'LG', '투수', '1997-03-08', '188cm,', '100kg', '홍연초(마포구리틀)-홍은중-선린인터넷고'], ['109', '김대현', 'LG', '투수', '2001-01-05', '188cm,', '96kg', '서화초-석남중-성지고'], ['16', '김민성', 'LG', '내야수', '1988-12-17', '181cm,', '94kg', '고명초-잠신중-덕수정보고-(영남사이버대)-롯데-넥센-키움'], ['5', '김용의', 'LG', '내야수', '1985-08-20', '187cm,', '74kg', '사당초-선린중-선린인터넷고-고려대-두산'], ['106', '김유민', 'LG', '내야수', '2003-01-18', '183cm,', '80kg', '화곡초-영남중-덕수고'], ['57', '김윤식', 'LG', '투수'

In [32]:
WO_player = Player("WO")
WO_URL = URL(WO_player)
WO_info = player_info(WO_URL)
WO = to_DF(WO_player, WO_info)

78
[['65', '김동욱', '고양', '투수', '1997-05-16', '178cm,', '89kg', '(용산구리틀)-경원중-휘문고-홍익대'], ['98', '김동은', '고양', '투수', '2000-10-16', '186cm,', '100kg', '광주수창초-충장중-광주동성고'], ['60', '김동혁', '키움', '투수', '2001-12-27', '183cm,', '83kg', '(강남구리틀)-영동중-덕수고'], ['4', '김병휘', '키움', '내야수', '2001-02-16', '177cm,', '79kg', '효제초-홍은중-장충고'], ['49', '김선기', '키움', '투수', '1991-09-01', '187cm,', '94kg', '석교초-세광중-세광고-상무-넥센'], ['21', '김성민', '키움', '투수', '1994-04-26', '181cm,', '90kg', '대구옥산초-경복중-대구상원고-일본경제대-SK-넥센'], ['34', '김성진', '키움', '투수', '1997-11-14', '182cm,', '78kg', '율하초-포항제철중-부산정보고-계명대'], ['46', '김수환', '키움', '내야수', '1998-03-20', '180cm,', '100kg', '부일초(부평구리틀)-재능중-제물포고-넥센'], ['03', '김시앙', '고양', '포수', '2001-10-31', '176cm,', '83kg', '광주대성초-광주동성중-광주동성고'], ['58', '김신회', '키움', '외야수', '1999-07-14', '177cm,', '75kg', '둔촌초-배재중-제물포고'], ['1', '김웅빈', '키움', '내야수', '1996-02-09', '181cm,', '84kg', '서라벌초-제일중-울산공고-넥센-상무'], ['37', '김은성', '키움', '내야수', '1993-03-07', '179cm,', '83kg', '금평초-자양중-전주고-원광대'], ['67', '김인범', '고양', '투수', '

In [35]:
HT_player = Player("HT")
HT_URL = URL(HT_player)
HT_info = player_info(HT_URL)
HT = to_DF(HT_player, HT_info)

81
[['60', '고영창', 'KIA', '투수', '1989-02-24', '189cm,', '100kg', '광주서림초-진흥중-진흥고-연세대'], ['40', '권혁경', 'KIA', '포수', '2002-01-23', '187cm,', '94kg', '중대초-잠신중-신일고'], ['14', '김규성', 'KIA', '내야수', '1997-03-08', '183cm,', '73kg', '갈산초-선린중-선린인터넷고'], ['47', '김명찬', 'KIA', '투수', '1992-10-24', '187cm,', '106kg', '백마초-덕수중-선린고-연세대-KIA-경찰'], ['24', '김민식', 'KIA', '포수', '1989-06-28', '180cm,', '80kg', '양덕초-마산중-마산고-원광대-SK-상무-SK'], ['3', '김선빈', 'KIA', '내야수', '1989-12-18', '165cm,', '77kg', '화순초-화순중-화순고-KIA-상무'], ['044', '김선우', 'KIA', '포수', '2001-01-18', '178cm,', '80kg', '상인천초-상인천중-강릉고'], ['013', '김양수', 'KIA', '투수', '2001-01-23', '184cm,', '80kg', '천안남산초-천안북중-북일고'], ['07', '김연준', 'KIA', '외야수', '1996-12-20', '180cm,', '90kg', '중대초-잠신중-경기고-중앙대'], ['64', '김영환', 'KIA', '내야수', '1993-01-31', '178cm,', '74kg', '남도초-자양중-신일고-(서울사이버대)-KT-경찰-KT'], ['66', '김원경', 'KIA', '내야수', '2001-05-01', '182cm,', '83kg', '사파초-마산중-비봉고'], ['49', '김유신', 'KIA', '투수', '1999-06-14', '187cm,', '100kg', '화순초-청주중-세광고-KIA-상무'], ['28', '김윤동',

In [37]:
LT_player = Player("LT")
LT_URL = URL(LT_player)
LT_info = player_info(LT_URL)
LT = to_DF(LT_player, LT_info)

73
[['66', '강동호', '롯데', '투수', '1994-04-21', '189cm,', '118kg', '신원초-영남중-배재고-원광대-롯데-상무'], ['1', '강로한', '롯데', '외야수', '1992-05-13', '181cm,', '78kg', '울산삼신초-경남중-부경고-경남대'], ['27', '강태율', '롯데', '포수', '1996-11-01', '180cm,', '88kg', '부산수영초-대천중-부경고'], ['22', '구승민', '롯데', '투수', '1990-06-12', '181cm,', '80kg', '동일초(도봉구리틀)-청원중-청원고-홍익대-롯데-상무'], ['97', '권동현', '롯데', '투수', '2001-08-12', '192cm,', '88kg', '부산연서초-센텀중-부경고'], ['24', '김강현', '롯데', '포수', '1995-02-27', '177cm,', '80kg', '고명초-청원중-청원고-(세계사이버대)'], ['42', '김건국', '롯데', '투수', '1988-02-02', '179cm,', '85kg', '한서초(서부리틀)-청량중-덕수정보고-두산-KT'], ['41', '김대우', '롯데', '투수', '1984-07-26', '189cm,', '95kg', '광주대성초-무등중-광주제일고-상무'], ['23', '김도규', '롯데', '투수', '1998-07-11', '190cm,', '98kg', '고봉초-충암중-안산공고'], ['37', '김동규', '롯데', '투수', '1999-04-08', '185cm,', '85kg', '안일초(안양시리틀)-현도중-포철고'], ['57', '김동우', '롯데', '투수', '1995-10-01', '184cm,', '76kg', '서당초-매송중-야탑고-연세대'], ['5', '김민수', '롯데', '내야수', '1998-03-18', '182cm,', '96kg', '서화초-동산중-제물포고-롯데-경찰'], ['34', '김원중', '롯데', '

In [39]:
SS_player = Player("SS")
SS_URL = URL(SS_player)
SS_info = player_info(SS_URL)
SS = to_DF(SS_player, SS_info)

81
[['47', '강민호', '삼성', '포수', '1985-08-18', '185cm,', '100kg', '제주신광초-포철중-포철공고-(국제디지털대)-롯데'], ['6', '강한울', '삼성', '내야수', '1991-09-12', '181cm,', '66kg', '사당초-중앙중-안산공고-원광대-KIA-삼성-상무'], ['5', '구자욱', '삼성', '외야수', '1993-02-12', '189cm,', '75kg', '본리초-경복중-대구고-삼성-상무'], ['19', '구준범', '삼성', '투수', '1995-03-18', '175cm,', '68kg', '서울학동초-대치중-배명고'], ['48', '권정웅', '삼성', '포수', '1992-11-15', '180cm,', '88kg', '서울이수초-영동중-덕수고-한양대-삼성-상무'], ['102', '김결의', '삼성', '포수', '1994-01-03', '179cm,', '90kg', '노암초-경포중-강릉고-성균관대'], ['105', '김경민', '삼성', '내야수', '1997-08-01', '183cm,', '100kg', '진북초-전라중-포철고-성균관대'], ['17', '김대우', '삼성', '투수', '1988-11-21', '183cm,', '85kg', '역삼초-대치중-서울고-홍익대-넥센-상무-넥센'], ['42', '김도환', '삼성', '포수', '2000-04-14', '178cm,', '90kg', '언북초(의정부리틀)-영동중-신일고'], ['38', '김동엽', '삼성', '외야수', '1990-07-24', '186cm,', '101kg', '천안남산초-천안북중-북일고-SK'], ['103', '김동진', '삼성', '내야수', '1996-12-18', '184cm,', '84kg', '영랑초-설악중-설악고'], ['106', '김동찬', '삼성', '투수', '1999-04-12', '179cm,', '92kg', '대룡초(동구리틀)-청량중-덕수고'], ['12',

In [41]:
SK_player = Player("SK")
SK_URL = URL(SK_player)
SK_info = player_info(SK_URL)
SK = to_DF(SK_player, SK_info)

82
[['99', '강지광', 'SSG', '투수', '1990-10-23', '181cm,', '100kg', '진북초-상인천중-인천고-LG-넥센-SK'], ['68', '고명준', 'SSG', '내야수', '2002-07-08', '185cm,', '94kg', '서원초-세광중-세광고-SK'], ['38', '고종욱', 'SSG', '외야수', '1989-01-11', '184cm,', '83kg', '역삼초-대치중-경기고-한양대-넥센-상무-넥센-SK'], ['07', '권혁찬', 'SSG', '내야수', '1998-08-02', '189cm,', '97kg', '팔곡초(안산리틀)-소래중-인천고-홍익대-SK'], ['61', '길지석', 'SSG', '투수', '2001-11-02', '190cm,', '110kg', '(미추홀구리틀)-매송중-야탑고-SK'], ['0', '김강민', 'SSG', '외야수', '1982-09-13', '182cm,', '87kg', '본리초-대구중-경북고-SK'], ['67', '김건우', 'SSG', '투수', '2002-07-12', '185cm,', '88kg', '가현초(인천서구리틀)-동산중-제물포고-SK'], ['56', '김경호', 'SSG', '외야수', '1995-07-31', '180cm,', '68kg', '희망대초-매송중-야탑고-두산-SK'], ['49', '김민재', 'SSG', '외야수', '1996-04-04', '191cm,', '95kg', '연현초-양천중-안산공고-SK'], ['31', '김상수', 'SSG', '투수', '1988-01-02', '180cm,', '88kg', '신자초(자이언츠리틀)-자양중-신일고-(방송통신대)-삼성-넥센-상무-넥센-키'], ['5', '김성민', 'SSG', '내야수', '2001-04-30', '184cm,', '88kg', '서울학동초-자양중-경기고-SK'], ['16', '김성현', 'SSG', '내야수', '1987-03-09', '172cm,', '

In [43]:
HH_player = Player("HH")
HH_URL = URL(HH_player)
HH_info = player_info(HH_URL)
HH = to_DF(HH_player, HH_info)

79
[['7', '강경학', '한화', '내야수', '1992-08-11', '180cm,', '72kg', '광주대성초-광주동성중-광주동성고'], ['19', '강상원', '한화', '외야수', '1997-05-05', '172cm,', '64kg', '온양온천초-온양중-북일고'], ['55', '강재민', '한화', '투수', '1997-04-03', '180cm,', '86kg', '양덕초-마산중-용마고-단국대'], ['65', '권용우', '한화', '투수', '1993-03-17', '183cm,', '80kg', '도신초-성남중-성남고-동의대'], ['03', '김규연', '한화', '투수', '2002-08-23', '183cm,', '83kg', '동수원초(수원영통리틀)-매향중-공주고'], ['15', '김기중', '한화', '투수', '2002-11-16', '186cm,', '94kg', '의왕부곡초-매송중-유신고'], ['29', '김기탁', '한화', '투수', '1998-09-30', '182cm,', '82kg', '분성초(장유리틀)-내동중-김해고'], ['53', '김민우', '한화', '투수', '1995-07-25', '189cm,', '105kg', '사파초-마산중-용마고'], ['39', '김민하', '한화', '외야수', '1989-02-25', '181cm,', '88kg', '승학초-대신중-경남고-중앙대-롯데-상무-롯데'], ['47', '김범수', '한화', '투수', '1995-10-03', '182cm,', '81kg', '온양온천초-온양중-북일고'], ['94', '김승일', '한화', '투수', '2001-07-07', '183cm,', '85kg', '해강초(해운대리틀)-센텀중-경남고'], ['45', '김이환', '한화', '투수', '2000-09-15', '183cm,', '87kg', '길원초(동대문구리틀)-잠신중-신일고'], ['1', '김종수', '한화', '투수', '1994-06-03', '18

In [111]:
kbo_player = pd.concat([NC,OB,KT,LG,WO,HT,LT,SS,SK,HH])
kbo_player['PlayerID'] = kbo_player['팀'] + '-' + kbo_player['number']
kbo_player['debutYear'] = np.where(kbo_player['debut'].str.len() == 0, '','20'+kbo_player['debut'].str[:2])
kbo_player['debutYear'] = np.where(kbo_player['debutYear'].str[2:] > '90', '19'+kbo_player['debut'].str[:2], kbo_player['debutYear'])
kbo_player['debutTeam'] = kbo_player['debut'].str[2:]
kbo_player = kbo_player.drop(['key','weight','url','debut'], axis=1)
kbo_player.number = kbo_player.number.astype ("str")
kbo_player.to_csv('kbo_player.csv',encoding='utf-8-sig', index = False)

In [112]:
#Result
kbo_player

,number,이름,팀,pos,birth,league,tuta_type,payment,salary,PlayerID,debutYear,debutTeam
0,21,강동연,NC,투수,1992-12-18,1,우투우타,,4400만원,NC-21,2012,두산
1,68,강윤구,NC,투수,1990-07-10,1,좌투좌타,12000만원,10000만원,NC-68,2009,히어로즈
2,49,강진성,NC,내야수,1993-10-19,1,우투우타,7000만원,12000만원,NC-49,2012,NC
3,78,강태경,NC,투수,2001-07-26,2,우투좌타,7000만원,3000만원,NC-78,,
4,59,구창모,NC,투수,1997-02-17,1,좌투좌타,15000만원,25000만원,NC-59,2015,NC
...,...,...,...,...,...,...,...,...,...,...,...,...
74,16,하주석,한화,내야수,1994-02-25,1,우투좌타,30000만원,13500만원,한화-16,2012,한화
75,59,한승주,한화,투수,2001-03-17,1,우투우타,11000만원,3000만원,한화-59,2020,한화
76,26,허관회,한화,포수,1999-02-12,1,우투우타,3000만원,3100만원,한화-26,2020,한화
77,60,황영국,한화,투수,1995-12-26,1,좌투좌타,20000만원,3300만원,한화-60,2014,한화
